In [26]:
import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt

hecateFile = pd.read_csv('/home/db/Desktop/AstroProject/Data/hecate_v1.1_for_GW.csv')
fitsFile = '/home/db/Desktop/AstroProject/Data/MCMC_TF2_LowSpin_AllSky.fits'

[prob, distmu, distsigma, distnorm], header = hp.read_map(fitsFile, field=[0, 1, 2, 3], h = True) # creates 2 arrays, one with 4 columns, and one for the header column that is needed to find an abstract value which is the NSIDE for the ang2pix function

headerDict = {nameKeys:value for nameKeys,value in header} # Creates a dictionnary between a key and a value associated

HECATE_pixelsIndex = hp.ang2pix(headerDict['NSIDE'], hecateFile['RA'], np.array(hecateFile['DEC']), lonlat=True) # Converts the 2 angle values we have to pixel values, that we'll then make correspond with the FITS pixels

In [27]:
# P2D top 5 sources

P2D = prob[HECATE_pixelsIndex] # P2D probability associated to all galaxy of HECATE

sumProb = np.sum(P2D)

P2D_norm = P2D/sumProb # Normalized P2D array

fiveMaxP2DIndex = np.argsort(P2D_norm)[-5:][::-1] # takes the 5 largest elements sorted from largest to smallest ([-5:] is from smallest to largest only)

fiveMaxP2D = P2D_norm[fiveMaxP2DIndex] # fiveMaxProb is the top 5 galaxy candidates that can be the source of the GW

fiveMaxP2DNames = hecateFile['OBJNAME'][fiveMaxP2DIndex]

fiveMaxP2DNames = fiveMaxP2DNames.to_numpy()

print(np.c_[fiveMaxP2DNames, fiveMaxP2D])

[['PGC169672' 0.006675970091842589]
 ['PGC831458' 0.00656682202026065]
 ['PGC158506' 0.006526787685675716]
 ['PGC837800' 0.0063802623433190314]
 ['2MASXJ13042517-2042527' 0.0062625413769155265]]


In [28]:
# Variables

P_pixel = P2D # Renaming the non-normalized P2D for all galaxies

N_pixel = distnorm[HECATE_pixelsIndex]

D_galaxy = hecateFile['D'].to_numpy()

mu_pixel = distmu[HECATE_pixelsIndex]

mu_pixel = np.array([x if x >= 0 else 0 for x in mu_pixel]) # changes all negative values of mu_pixel to 0

sig_pixel = distsigma[HECATE_pixelsIndex]

exponent = -1/2*((D_galaxy-mu_pixel)/sig_pixel)**2

GaussFactor = np.exp(exponent) # Gaussian exponential of (1) of Ducoin paper

In [29]:
# P3D top 5 sources

P3D = P_pixel*N_pixel*GaussFactor # Formula (1) of the Ducoin paper

P3D = np.nan_to_num(P3D) # sets all nan values to 0

sumP3D = np.sum(P3D)

P3D_norm = P3D/sumP3D # Normalized P3D array

fiveMaxP3DIndex = np.argsort(P3D_norm)[-5:][::-1] 

fiveMaxP3D = P3D_norm[fiveMaxP3DIndex]

fiveMaxP3DNames = hecateFile['OBJNAME'][fiveMaxP3DIndex]

fiveMaxP3DNames = fiveMaxP3DNames.to_numpy()

print(np.c_[fiveMaxP3DNames, fiveMaxP3D])

[['ESO508-004' 0.0964901982774173]
 ['ESO575-053' 0.08491642944762096]
 ['NGC4993' 0.08356755221946364]
 ['ESO575-055' 0.08168905970345927]
 ['ESO508-024' 0.06740727980139116]]


In [30]:
# P3D*n(M*) top 5 sources

Smass = hecateFile['logM_HEC'].to_numpy() # Column logM_HEC from the csv file

PnM = P3D*np.exp(Smass) # Using formula (1) of the Artale paper

PnM = np.nan_to_num(PnM)

sumPnM = np.sum(PnM)

PnM = PnM/sumPnM # Normalized PnM array

fiveMaxPnMIndex = np.argsort(PnM)[-5:][::-1]

fiveMaxPnM = PnM[fiveMaxPnMIndex]

fiveMaxPnMnames = hecateFile['OBJNAME'][fiveMaxPnMIndex]

fiveMaxPnMnames = fiveMaxPnMnames.to_numpy()

print(np.c_[fiveMaxPnMnames, fiveMaxPnM])

[['NGC4993' 0.2749947510338346]
 ['ESO575-053' 0.11071554858557876]
 ['ESO508-024' 0.10330127491740833]
 ['ESO575-055' 0.0828156275139597]
 ['NGC4968' 0.07317182612806192]]


In [31]:
# P3D*n(M*,SFR) top 5 sources

SFR = (hecateFile['logSFR_HEC'].to_numpy())

PnMSFR = P3D*np.exp(Smass+SFR) # Using formula (2) of the Artale paper

PnMSFR = np.nan_to_num(PnMSFR)

sumPnMSFR = np.sum(PnMSFR)

PnMSFR = PnMSFR/sumPnMSFR # Normalized PnMSFR array

fiveMaxPnMSFRIndex = np.argsort(PnMSFR)[-5:][::-1]

fiveMaxPnMSFR = PnMSFR[fiveMaxPnMSFRIndex]

fiveMaxPnMSFRnames = hecateFile['OBJNAME'][fiveMaxPnMSFRIndex]

fiveMaxPnMSFRnames = fiveMaxPnMSFRnames.to_numpy()

print(np.c_[fiveMaxPnMSFRnames, fiveMaxPnMSFR])

[['NGC4968' 0.3418277461856768]
 ['ESO508-024' 0.20197545983987142]
 ['ESO575-053' 0.17182229479880456]
 ['ESO508-010' 0.11144259958750834]
 ['IC4180' 0.08091132906860676]]


In [32]:
# Results

print("P2D", np.c_[fiveMaxP2DNames, fiveMaxP2D], " "
      , "P3D", np.c_[fiveMaxP3DNames, fiveMaxP3D], " "
      , "P3D*n(M*)", np.c_[fiveMaxPnMnames, fiveMaxPnM], " "
      , "P3D*n(M*,SFR)", np.c_[fiveMaxPnMSFRnames, fiveMaxPnMSFR], sep='\n')

P2D
[['PGC169672' 0.006675970091842589]
 ['PGC831458' 0.00656682202026065]
 ['PGC158506' 0.006526787685675716]
 ['PGC837800' 0.0063802623433190314]
 ['2MASXJ13042517-2042527' 0.0062625413769155265]]
 
P3D
[['ESO508-004' 0.0964901982774173]
 ['ESO575-053' 0.08491642944762096]
 ['NGC4993' 0.08356755221946364]
 ['ESO575-055' 0.08168905970345927]
 ['ESO508-024' 0.06740727980139116]]
 
P3D*n(M*)
[['NGC4993' 0.2749947510338346]
 ['ESO575-053' 0.11071554858557876]
 ['ESO508-024' 0.10330127491740833]
 ['ESO575-055' 0.0828156275139597]
 ['NGC4968' 0.07317182612806192]]
 
P3D*n(M*,SFR)
[['NGC4968' 0.3418277461856768]
 ['ESO508-024' 0.20197545983987142]
 ['ESO575-053' 0.17182229479880456]
 ['ESO508-010' 0.11144259958750834]
 ['IC4180' 0.08091132906860676]]
